In [69]:
# Задание 1

# Используем файл keywords.csv.

# Необходимо написать гео-классификатор, 
# который каждой строке сможет выставить географическую принадлежность определенному региону. 
# Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. 
# Если поисковый запрос не содержит названия города, то ставим 'undefined'.

# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
# Результат классификации запишите в отдельный столбец region.

import pandas as pd

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

keywords_df = pd.read_csv('keywords.csv')

def region_func(row):
     
    # Проверка keywords_df['keyword'] по geo_data => сопоставление региона 
    #    keywords_df['keyword'] может быть многословной строкой через пробел => создаем список linelist 
    # Внешний цикл по списку linelist
    # Внутренний цикл по словарю geo_data => Если слово в linelist ('москва') in value (['москва', 'тула', 'ярославль']), вернуть key ('Центр')

    index_geo = 'undefined'
    linelist = row['keyword'].split(' ')  
    
    for line in linelist:
        for key, value in geo_data.items():
            if line in value : 
                index_geo = key
                return index_geo
                
    return index_geo

# Искомый Гео-классификатор
keywords_df['region'] = keywords_df.apply(region_func, axis=1)

# Проверим гео-классификатор - для наглядности по фильтру регионов
filtered_keywords = keywords_df[ (keywords_df['region']=='Центр') | (keywords_df['region']=='Северо-Запад') | (keywords_df['region']=='Дальний Восток') ]
filtered_keywords.head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


In [70]:
# Задание 2

# Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

#     оценка 2 и меньше - низкий рейтинг
#     оценка 4 и меньше - средний рейтинг
#     оценка 4.5 и 5 - высокий рейтинг
# Результат классификации запишите в столбец class

import pandas as pd

ratings_df = pd.read_csv('ml-latest-small/ratings.csv')

def class_func(row):
    i = row['rating']
    
    if i <= 2:
        return_class = 'низкий'
    elif i <= 4:
        return_class = 'средний'
    else:
       return_class = 'высокий'

    return return_class
    
# Считаем средний рейтинг фильмов            
movies_class = ratings_df.groupby('movieId').mean()[['rating']].reset_index() 

# Классифицируем. Результат классификации запишим в столбец class
movies_class['class'] = movies_class.apply(class_func, axis=1)

# Посмотрим top-фильмы с рейтингом по убыванию
movies_class.sort_values('rating', ascending=False).head()


,movieId,rating,class
9065,163949,5.0,высокий
7297,71180,5.0,высокий
6629,51471,5.0,высокий
6662,52617,5.0,высокий
6704,53887,5.0,высокий


In [71]:
# Задание 3

# Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
# Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
# Ответ дайте в днях.

import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')

# Сгруппируем по пользователям
ratings_filtered = ratings.groupby('userId').count()

# Отфильтруем по кол-ву оценок (по кол-ву строк датасета), поэтому можно ставить любой столбец, например timestamp, и почистим индекс
ratings_filtered = ratings_filtered[ratings_filtered['timestamp'] > 100].reset_index()

# Получить оригинальные рейтинги (из таблицы ratings) для пользователей которые поставили больше 100 оценок.
top_user = ratings[ ratings['userId'].isin( ratings_filtered['userId'].tolist() ) ]

# Посчитать Lifetime киноманов
lifetime_top_user = top_user.groupby('userId').agg([min, max])
lifetime_top_user['diff'] = lifetime_top_user['timestamp']['max'] - lifetime_top_user['timestamp']['min']

# Считаем среднее значение Lifetime всех киноманов
lifetime_top_user['diff'].mean() / 24 / 3600


463.89476214829165

In [73]:
# Задание 4

# Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". 
# Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. 
# Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

# При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

#     В переменную years запишите список из всех годов с 1950 по 2010.
#     Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
#     Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

#   - для каждой строки пройдите по всем годам списка years
#   - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
#   - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

#     Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
#     Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

import pandas as pd

movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# Функция production_year, которая каждой строке из названия фильма выставляет год выпуска.    
def production_year(row):
    years = list( range(1950, 2011) )
    result_year = 1900
    
    for y in years:
        if str(y) in (row['title']):
            result_year = str(y)
    return result_year

# Проставляем год выпуска фильма по алгоритму production_year в новый столбец 'year'
movies['year'] = movies.apply(production_year, axis=1)

# Join фильмов c рейтингами по полю movieId (чтобы подтащить 'year' к рейтингам)
joined_mr = ratings.merge(movies, on='movieId', how='left')

# Считаем средний рейтинг фильмов для каждого значения столбца 'year', сортируем результат по убыванию рейтинга
avg_rating_years = joined_mr.groupby('year').mean()[['rating']].reset_index().sort_values('rating', ascending=False) 
avg_rating_years.head()


,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
25,1974,3.999058
5,1954,3.994220
